<a href="https://colab.research.google.com/github/Adtestdhahri/STRAVA2/blob/main/5tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# 2. Install required packages
!pip install requests pandas openpyxl --quiet

# 3. Import libraries
import requests
import pandas as pd
import numpy as np
import os

# 4. Refresh Strava access token
client_id = "161942"
client_secret = "b4344d82acf8d8bbe0b4ffbb09bcc478909d3d71"
refresh_token = "87150902248e0dcd3ecdab6f41863c1a7882b050"

response = requests.post(
    "https://www.strava.com/oauth/token",
    data={
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "refresh_token",
        "refresh_token": refresh_token
    }
)
tokens = response.json()
access_token = tokens["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}

# 5. Fetch club activities
url = "https://www.strava.com/api/v3/clubs/491970/activities"
response = requests.get(url, headers=headers)
data = response.json()

# 6. Convert to DataFrame
df = pd.json_normalize(data)

# 7. Compute derived metrics
df["efficiency_ratio"] = df["moving_time"] / df["elapsed_time"]
df["pace_sec_per_km"] = df["moving_time"] / (df["distance"].replace(0, np.nan) / 1000)
df["speed_kmh"] = (df["distance"] / 1000) / (df["moving_time"] / 3600)
df["intensity_score"] = (
    (df["distance"] / 1000)
    + (df["total_elevation_gain"] / 10)
    + (df["moving_time"] / 60)
)

# 8. Build summary tables
athlete_summary = df.groupby(
    ["athlete.firstname", "athlete.lastname", "sport_type"]
).agg({
    "distance": "sum",
    "moving_time": "sum",
    "elapsed_time": "sum",
    "total_elevation_gain": "sum"
}).reset_index()

activity_count = df.groupby(
    ["athlete.firstname", "athlete.lastname", "sport_type"]
).size().reset_index(name="activity_count")

pace_summary = df.groupby("sport_type")[["pace_sec_per_km", "speed_kmh"]].mean().reset_index()

top_intense = df.sort_values("intensity_score", ascending=False).head(10)

leaderboard = df.groupby(
    ["athlete.firstname", "athlete.lastname"]
).agg({
    "distance": "sum",
    "moving_time": "sum",
    "intensity_score": "sum"
}).reset_index().sort_values("intensity_score", ascending=False)
leaderboard["fullname"] = leaderboard["athlete.firstname"] + " " + leaderboard["athlete.lastname"]

# 9. Define Drive folder and ensure it exists
drive_folder = "/content/drive/MyDrive/StravaProjectData"
os.makedirs(drive_folder, exist_ok=True)

# 10. Write each summary table to its own Excel file
athlete_summary.to_csv(
    os.path.join(drive_folder, "athlete_summary.csv"),
    index=False
)

activity_count.to_csv(
    os.path.join(drive_folder, "activity_count.csv"),
    index=False
)

pace_summary.to_csv(
    os.path.join(drive_folder, "pace_summary.csv"),
    index=False
)

top_intense.to_csv(
    os.path.join(drive_folder, "top_intense.csv"),
    index=False
)

leaderboard.to_csv(
    os.path.join(drive_folder, "leaderboard.csv"),
    index=False
)

print("✅ Each summary table has been written as a separate Excel file under:")
print(f"   {drive_folder}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Each summary table has been written as a separate Excel file under:
   /content/drive/MyDrive/StravaProjectData


In [ ]:
# 1. Mount Google Drive (if in Colab)
from google.colab import drive
drive.mount('/content/drive')

# 2. Install required packages
!pip install gspread pandas --quiet

# 3. Authenticate your Google user (runs a small OAuth flow)
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# 4. Import pandas & numpy for DataFrame handling
import pandas as pd
import numpy as np
import os

# 5. Define the folder where your CSVs already exist
drive_folder = "/content/drive/MyDrive/StravaProjectData"

# 6. Paths to each summary CSV (adjust names if yours differ)
paths = {
    "Athlete_Summary": os.path.join(drive_folder, "athlete_summary.csv"),
    "Activity_Count":   os.path.join(drive_folder, "activity_count.csv"),
    "Pace_Summary":     os.path.join(drive_folder, "pace_summary.csv"),
    "Top_Intense":      os.path.join(drive_folder, "top_intense.csv"),
    "Leaderboard":      os.path.join(drive_folder, "leaderboard.csv"),
}

# 7. Read each CSV into a DataFrame, sanitize NaN/inf, and collect
dfs = {}
for sheet_name, csv_path in paths.items():
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"Could not find {csv_path}")
    df = pd.read_csv(csv_path)
    # Replace NaN or infinite values with empty strings:
    df = df.replace([np.inf, -np.inf], np.nan).fillna("")
    # Convert numpy numeric types to native Python types:
    for col in df.columns:
        if pd.api.types.is_integer_dtype(df[col].dtype) or pd.api.types.is_float_dtype(df[col].dtype):
            df[col] = df[col].astype(object).where(df[col] != "", df[col])
    dfs[sheet_name] = df

# 8. Helper to write a DataFrame to a new Google Sheet
def df_to_new_sheet(df: pd.DataFrame, title: str) -> str:
    """
    Creates a Google Sheet named `title`, writes df into its first worksheet,
    and returns the URL of the new sheet.
    """
    sh = gc.create(title)
    ws = sh.sheet1
    ws.update_title(title[:30].replace(" ", "_"))  # rename tab to a valid name

    # Prepare header + rows
    headers = df.columns.tolist()
    rows    = df.values.tolist()
    data    = [headers] + rows

    # Resize and batch‐write
    n_rows = len(data)
    n_cols = len(headers)
    ws.resize(rows=n_rows, cols=n_cols)
    cell_list = ws.range(1, 1, n_rows, n_cols)
    flat_vals = [item for sublist in data for item in sublist]
    for i, val in enumerate(flat_vals):
        cell_list[i].value = val
    ws.update_cells(cell_list)

    return sh.url

# 9. Create one sheet per summary table
urls = {}
for name, df in dfs.items():
    url = df_to_new_sheet(df, name)
    urls[name] = url

# 10. Print out the sheet URLs
print("✅ Created the following Google Sheets from CSVs:")
for name, url in urls.items():
    print(f"• {name}: {url}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Created the following Google Sheets from CSVs:
• Athlete_Summary: https://docs.google.com/spreadsheets/d/12YAVJMzY7EFYZ7_n6yOqHMIr-4Bn6pLIiqt5vE2P5wQ
• Activity_Count: https://docs.google.com/spreadsheets/d/1yMrpZJ1y7ShoiwzJulFiYK0H1XzVWk-WWhGxCqyEQJo
• Pace_Summary: https://docs.google.com/spreadsheets/d/1gP6hmCuLGcovfewT69RybxdlF3lkmcTuYyMAMUUDMUU
• Top_Intense: https://docs.google.com/spreadsheets/d/1KWTtMigiPW8cYQkOrco-mgBIMqn5GsXLT5W5ZV0sQ68
• Leaderboard: https://docs.google.com/spreadsheets/d/1lQdNtfbfU__FoT7u42Oe3UILTWDkBateo_XLdF7bh0M
